<a href="https://colab.research.google.com/github/mtszkw/kaggle-lish-moa/blob/main/kaggle_lish_moa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
input_dir = '/content/'
# input_dir = '../input/lish-moa'

In [2]:
# https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter

import os
import sys

!pip -q install --upgrade kaggle

%env KAGGLE_USERNAME=mtszkw
%env KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41

# Dataset
# https://www.kaggle.com/c/lish-moa
!kaggle competitions download -c lish-moa

# iterative-stratification
# https://www.kaggle.com/yasufuminakama/iterative-stratification
!kaggle datasets download -d yasufuminakama/iterative-stratification

!unzip -q '/content/*.zip'

for x in os.listdir('/content'):
    if x.endswith(".zip"):
        os.remove(os.path.join('/content', x))

os.listdir('/content')

env: KAGGLE_USERNAME=mtszkw
env: KAGGLE_KEY=4542560feb261d28dd30710ffc95cb41
 98% 9.00M/9.16M [00:00<00:00, 14.3MB/s]
100% 9.16M/9.16M [00:00<00:00, 11.1MB/s]
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 40.0MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 75.0MB/s]
 86% 47.0M/54.8M [00:00<00:00, 33.7MB/s]
100% 54.8M/54.8M [00:00<00:00, 62.7MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 76.8MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 76.8MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 18.6MB/s]

6 archives were successfully processed.


['.config',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'iterative-stratification-master',
 'test_features.csv',
 'train_features.csv',
 'train_drug.csv',
 'sample_submission.csv',
 'sample_data']

In [10]:
import numpy as np
import pandas as pd
from typing import Optional

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip -q install pytorch-lightning
import pytorch_lightning as pl

!pip -q install '/content/iterative-stratification-master'
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [11]:
train_features = pd.read_csv(os.path.join(input_dir, 'train_features.csv'))
train_targets_scored = pd.read_csv(os.path.join(input_dir, 'train_targets_scored.csv'))
# train_targets_nonscored = pd.read_csv(os.path.join(input_dir, 'train_targets_nonscored.csv'))
test_features = pd.read_csv(os.path.join(input_dir, 'test_features.csv'))
# submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))

print(f'Train: {train_features.shape}, test: {test_features.shape}')

Train: (23814, 876), test: (3982, 876)


### Dataset

In [15]:
class MoATrainDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
class MoATestDataset(torch.utils.data.Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        X = self.features[idx]
        return X

In [16]:
class MoADataModule(pl.LightningDataModule):
    def __init__(self, batch_size, train_ds, valid_ds, test_ds):
        super().__init__()
        self.batch_size = batch_size
        self.train_ds = train_ds
        self.valid_ds = valid_ds
        self.test_ds = test_ds

    def setup(self, stage: Optional[str] = None):
        if stage == 'fit' or stage is None:
            pass
        if stage == 'test' or stage is None:
            pass

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_ds, shuffle=False, batch_size=self.batch_size)

    def valid_dataloader(self):
        return torch.utils.data.DataLoader(
            self.valid_ds, shuffle=False,  batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_ds, shuffle=False, batch_size=self.batch_size)
        

### Model

In [19]:
class MoANeuralNetwork(pl.LightningModule):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = self.get_model(input_size, output_size)

    def get_model(self, input_size, output_size):
        dropout = .5
        hidden_size = 512

        return nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            nn.PReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            nn.PReLU(),
            nn.Linear(hidden_size, output_size))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=0.01, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        return [optimizer], [scheduler]

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({'loss': loss})
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = torch.nn.BCEWithLogitsLoss()(y_hat, y)

        self.log_dict({'val_loss': loss})
        return loss


### Training

In [ ]:
target_cols = [c for c in train_targets_scored.columns if c not in ['sig_id']]
# print(len(target_cols))
# target_cols

# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
all_train = train_features.merge(train_targets_scored, on='sig_id')
all_train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
all_test  = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

print(f'Train: {all_train.shape}, test: {all_test.shape}')

In [ ]:
# folds = all_train.copy()

# k_fold = MultilabelStratifiedKFold(n_splits=5, shuffle=False, random_state=42)
# for n, (train_index, val_index) in enumerate(k_fold.split(folds, folds[target_cols])):

train_ds = MoATrainDataset(all_trainb)

    # folds.loc[val_index, 'fold'] = int(n)
# folds['fold'] = folds['fold'].astype(int)

# folds.head()

In [ ]:
trainer = pl.Trainer()

dataset = 
model = MoANeuralNetwork()

trainer.fit()